# MP 3: Grasp Planning

**Due date**: Feb 18, 2021 at 10:45am.

**Instructions**: Read and complete the problems below. To submit your assignment, perform the following:

1. Double-check that your notebook runs without error from a clean "Restart and Clear Output".
2. Download your completed notebook.  To avoid sending unnecessarily large notebooks, choose "Restart and Clear Output" before downloading. 
3. Submit your notebook (and any other new files you created) on Moodle [http:/learn.illinois.edu](http:/learn.illinois.edu).

In [ ]:
#Imports

#If you have wurlitzer installed, this will help you catch printouts from Klamp't
#Note: doesn't work on Windows
#%load_ext wurlitzer

import time
from klampt import *
from klampt import vis
from klampt.math import vectorops,so3,se3
from klampt.model.trajectory import Trajectory,SE3Trajectory
from klampt.model import ik
from klampt.io import numpy_convert
import numpy as np
import math
import random
import sys
sys.path.append('../common')
import gripper
import known_grippers
import normals
vis.init('IPython')
pass

In [ ]:
world = WorldModel()
obj = world.makeRigidObject("object1")
#obj.geometry().loadFile("../data/objects/ycb-select/002_master_chef_can/nontextured.ply")
#obj.geometry().loadFile("../data/objects/ycb-select/003_cracker_box/nontextured.ply")
#obj.geometry().loadFile("../data/objects/ycb-select/011_banana/nontextured.ply"); 
obj.geometry().loadFile("../data/objects/ycb-select/048_hammer/nontextured.ply")
#obj.geometry().loadFile("../data/objects/cube.off"); obj.geometry().scale(0.2)
#weird bug in Qhull -- cylinder can't be converted to ConvexHull
#obj.geometry().loadFile("../data/objects/cylinder.off")

#this will perform a reasonable center of mass / inertia estimate
m = obj.getMass()
m.estimate(obj.geometry(),mass=0.454,surfaceFraction=0.2)
obj.setMass(m)

#make the object transparent yellow
obj.appearance().setColor(0.8,0.8,0.2,0.5)
#draw center of mass
vis.createWindow()
closeup_viewport = closeup_viewport = {'up': {'z': 0, 'y': 1, 'x': 0}, 'target': {'z': 0, 'y': 0, 'x': 0}, 'near': 0.1, 'position': {'z': 1.0, 'y': 0.5, 'x': 0.0}, 'far': 1000}
vis.setViewport(closeup_viewport)
vis.add("world",world)
vis.add("COM",m.getCom(),color=(1,0,0,1),size=0.01)
vis.show()

## Problem 1: Planning antipodal grasps

Problem 1 asks to find antipodal grasps for a given object geometry, that is, line segments that a gripper could plausibly use for closing onto the geometry.  Here we will use the `AntipodalGrasp` data structure to represent these grasps, as well as the `GripperInfo` data structure defined in `cs498ir_s2021/common/gripper.py`. Read the documentation to better understand the meaning of these members.

The key defining parameters of an antipodal grasp include the center of the line segment and its axis. From these parameters, you should be able to fill in its contact points and the grasp width using some geometric computations. 

A. First, you will fill in the `fill_in_grasp` function, which takes a center and axis and fills out the other members of the AntipodalGrasp data structure.  The [Geometry3D.rayCast_ext](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/klampt.robotsim.html#klampt.Geometry3D.rayCast_ext) method used in `ray_cast_to_contact_point` will be helpful for you to retrieve the normal and position of a contact point. The subsequent cell will show the resulting grasps in red if they have no contact point, and green if they do have a contact point.  Moreover, the endpoints of the grasp will be arranged slightly outside of the object.

In [ ]:
########################## Problem 1.A code goes in this cell ##############################

from klampt.model.contact import ContactPoint

#global data structure, will be filled in for you
object_normals = None

class AntipodalGrasp:
    """A structure containing information about antipodal grasps.
    
    Attributes:
        center (3-vector): the center of the fingers (object coordinates).
        axis (3-vector): the direction of the line through the
            fingers (object coordinates).
        approach (3-vector, optional): the direction that the fingers
            should move forward to acquire the grasp.
        finger_width (float, optional): the width that the gripper should
            open between the fingers.
        contact1 (ContactPoint, optional): a point of contact on the
            object.
        contact2 (ContactPoint, optional): another point of contact on the
            object.
    """
    def __init__(self,center,axis):
        self.center = center
        self.axis = axis
        self.approach = None
        self.finger_width = None
        self.contact1 = None
        self.contact2 = None

    def add_to_vis(self,name,color=(1,0,0,1)):
        finger_radius = 0.02
        if self.finger_width == None:
            w = 0.05
        else:
            w = self.finger_width*0.5+finger_radius
        a = vectorops.madd(self.center,self.axis,w)
        b = vectorops.madd(self.center,self.axis,-w)
        vis.add(name,[a,b],color=color)
        if self.approach is not None:
            vis.add(name+"_approach",[self.center,vectorops.madd(self.center,self.approach,0.05)],color=(1,0.5,0,1))
        normallen = 0.05
        if self.contact1 is not None:
            vis.add(name+"cp1",self.contact1.x,color=(1,1,0,1),size=0.01)
            vis.add(name+"cp1_normal",[self.contact1.x,vectorops.madd(self.contact1.x,self.contact1.n,normallen)],color=(1,1,0,1))
        if self.contact2 is not None:
            vis.add(name+"cp2",self.contact2.x,color=(1,1,0,1),size=0.01)
            vis.add(name+"cp2_normal",[self.contact2.x,vectorops.madd(self.contact2.x,self.contact2.n,normallen)],color=(1,1,0,1))

def grasp_from_contacts(contact1,contact2):
    """Helper: if you have two contacts, this returns an AntipodalGrasp"""
    d = vectorops.unit(vectorops.sub(contact2.x,contact1.x))
    grasp = AntipodalGrasp(vectorops.interpolate(contact1.x,contact2.x,0.5),d)
    grasp.finger_width = vectorops.distance(contact1.x,contact2.x)
    grasp.contact1 = contact1
    grasp.contact2 = contact2
    return grasp

def ray_cast_to_contact_point(obj,source,direction):
    """Produces a ContactPoint for the point hit by the ray:
        x(t) = source + t*direction
    
    Assumes object_normals is set up to match the object.
    
    Arguments:
        obj (RigidObjectModel)
        source (3-vector)
        direction (3-vector)
    
    Returns: ContactPoint with `x` set to the hit point,
    `n` set to the triangle normal, and coefficient of friction = 1.
    
    None is returned if the object is not hit by the ray.
    """
    assert object_normals is not None
    hit_tri,pt = obj.geometry().rayCast_ext(source,direction)
    if hit_tri < 0:
        return None
    return ContactPoint(pt,object_normals[hit_tri],1)

def fill_in_grasp(grasp,rigid_object):
    """TODO: you should fill this out to generate
    grasp.contact1, grasp.contact2 and grasp.finger_width.  
    
    Arguments:
        grasp (AntipodalGrasp): a partially-specified grasp (center and axis)
        rigid_object (RigidObjectModel): the object
    
    Returns None. (Just fill in the members of the grasp object.)
    """
    global object_normals
    if grasp.contact1 is not None:
        #assume already filled out
        return
    #put your code here
    return
    
###########################################################################################

In [ ]:
#Problem 1.A. Basic test of the fill-in method
vis.createWindow()
vis.setViewport(closeup_viewport)
vis.add("world",world)

grasp1 = AntipodalGrasp([0,0,0.02],[1,0,0])
grasp2 = AntipodalGrasp([0,0.2,0.02],[1,0,0])
grasp3 = AntipodalGrasp([0,0,0.02],[0,1,0])
grasps = [grasp1,grasp2,grasp3]
object_normals = normals.get_object_normals(obj)
for i,g in enumerate(grasps):
    name = "grasp{}".format(i)
    fill_in_grasp(g,obj)
    if g.contact1 is None:
        g.add_to_vis(name,(1,0,0,1)) 
    else:
        g.add_to_vis(name,(0,1,0,1)) 
vis.show()

B. Next, you will implement a grasp scoring technique.  Fill in the components of the `score` method to generate a scoring function for each antipodal grasp.  Your scoring function should implement at least 3 of the `within_fingers`, `centering_alignment`, `normal_alignment`, `off_axis_torque`, and `finger_clearance` features as described in their documentation.  The final score should weight between them. 

Read the documentation of the `GripperInfo` data structure to better understand the characteristics of the `self.gripper_info` object, which declares some of the key geometric parameters of the gripper. In particular, `maximum_span`, `finger_length`, `finger_width`, and `finger_depth` will be useful here.

In the second cell below, you will be given a good grasp and two bad grasps. Grasps that are obviously good should show up as green, and grasps that are obviously bad should show up as red.

In [ ]:
########################## Problem 1.B code goes in this cell ##############################

class GraspScore:
    """
    Your class for scoring grasps.
    
    Attributes:
        gripper_info (GripperInfo): see common/gripper_info.py
        rigid_object (RigidObjectModel):
        position_uncertainty (float): a value used to represent the
            amount of uncertainty we have in the gripper positioning.
        gravity (3-tuple): the gravity vector.
    """
    def __init__(self,gripper_info,rigid_object):
        self.gripper_info = gripper_info
        self.rigid_object = rigid_object
        global object_normals
        object_normals = normals.get_object_normals(rigid_object)
        self.position_uncertainty = 0.02
        self.gravity = (0,0,-9.8)

    def within_fingers(self,grasp):
        """Returns 1 if the object fits within gripper_info.maximum_span-position_uncertainty
        units along the specified grasp axis, and 0 if it has width greater than 
        gripper_info.maximum_span width, and otherwise decaying linearly from 1 to 0.

        You may want to use a helper function to fill in grasp.contact1 and
        grasp.contact2 if they aren't already there.
        """
        #TODO: your code here
        return 1

    def centering_alignment(self,grasp):
        """Returns the absolute error between the grasp center and the midpoint 
        between the points touched on the object.  If c is the center and
        m is the midpoint between the contact points p1 and p2, then this
        returns ||c-m||.

        You may want to use a helper function to fill in grasp.contact1 and
        grasp.contact2 if they aren't already there.
        """
        #TODO: your code here
        #fill_in_grasp(grasp,self.rigid_object)  #maybe use this?
        return 0

    def normal_alignment(self,grasp):
        """Returns the total angular error between the grasp axis and the normals
        of the object.  Specifically, returns angle(axis,n1) + angle(axis,n2) in
        radians, with n1 and n2 being the normals of the contact points.  (Choose
        the sign of axis that minimizes the angle.)

        You may want to use a helper function to fill in grasp.contact1 and
        grasp.contact2 if they aren't already there.
        """
        #TODO: your code here
        #fill_in_grasp(grasp,self.rigid_object)  #maybe use this?
        return 0

    def off_axis_torque(self,grasp):
        """Returns the magnitude of the torque imposed by gravity on the object
        for the given AntipodalGrasp.  (Don't forget to include the object's mass
        in your calculation).
        """
        #TODO: your code here
        return 0

    def finger_clearance(self,grasp):
        """Returns how much clearance the fingers have about the grasp.

        Imagine a point finger_radius away from the contact points, this
        should return min(d/gripper_info.finger_radius,1) where d is the minimum
        distance between the point and surrounding obstacles.
        """
        #fill_in_grasp(grasp,self.rigid_object)  #maybe use this?
        if grasp.contact1 is None: return 0
        return 1

    def score(self,grasp):
        """Returns a score indicating a quality of the grasp. Lower is better.
        """
        within = self.within_fingers(grasp)
        center = self.centering_alignment(grasp)
        normal = self.normal_alignment(grasp)
        off_axis = self.off_axis_torque(grasp)
        clearance = self.finger_clearance(grasp)
        #print(within,center,normal,off_axis,clearance)
        if within == 0: return float('inf')
        #TODO: weight between these components
        return (1.0-within) + center + normal + off_axis + (1.0-clearance)

###########################################################################################

In [ ]:
#Problem 1.B. Basic test of the quality of one obviously good grasp and two obviously bad ones.
#Feel free to debug your code by testing new grasps here
vis.createWindow()
vis.setViewport(closeup_viewport)
vis.add("world",world)

grasp1 = AntipodalGrasp([0,0,0.02],[1,0,0])
grasp2 = AntipodalGrasp([0,0.2,0.02],[1,0,0])
grasp3 = AntipodalGrasp([0,0,0.02],[0,1,0])
score = GraspScore(known_grippers.robotiq_140,obj)
grasps = [(g,score.score(g)) for g in [grasp1,grasp2,grasp3]]
for i,(g,s) in enumerate(grasps):
    name = "grasp{}".format(i)
    u = math.exp(-(s-grasps[0][1])*2)
    g.add_to_vis(name,(1-u,u,0,1)) 
vis.show()

C. Define a volume-centric grasp selection strategy in `antipodal_grasp_sample_volume` that samples grasps by sampling a random center point within the object's volume, and then samples a random direction for the axis.  From these N grasps, it should score and choose the top k grasps. The third cell below will show the results of your grasp sampler on whatever object was loaded in the initial cell at the top of this workbook.  By changing the object (try the banana and hammer!), you should observe different behavior.

In [ ]:
########################## Problem 1.C,D code goes in this cell ##############################

def antipodal_grasp_sample_volume(gripper,rigid_object,k,N):
    """Samples the top k high quality antipodal grasps for the
    object out of N sampled centers and axes within the object
    
    Returns a list of up to k (grasp,score) pairs, sorted in order
    of increasing score.
    """
    scorer = GraspScore(gripper,rigid_object)
    #this is a bounding box for the object
    bmin,bmax = rigid_object.geometry().getBBTight()
    grasps = []
    #TODO... add AntipodalGrasps to the grasps list
    scores = [(scorer.score(g),g) for g in grasps]
    scores = sorted(scores,key = lambda x:x[0])
    if k > len(grasps): k=len(grasps)
    return [(x[1],x[0]) for x in scores[:k]]

def sample_trimesh(verts,tris,N):
    global object_normals
    sample = [random.randint(0,len(verts)-1) for i in range(N)]
    pts = []
    normals = []
    for elem in sample:
        a,b,c = tris[elem]
        va = verts[a]
        vb = verts[b]
        vc = verts[c]
        u = random.random()
        v = random.random()
        if u+v > 1:
            v2 = (1-u)
            u2 = (1-v)
            u,v = u2,v2
        pt = va*(1-u-v) + vb*u + vc*v
        n = object_normals[elem]
        pts.append(pt)
        normals.append(n)
    return pts,normals

def antipodal_grasp_sample_surface(gripper,rigid_object,k,N):
    """Samples the top k high quality antipodal grasps for the
    object out of N sampled pairs of points on the surface of
    the object.
    
    Returns a list of up to k (grasp,score) pairs, sorted in order
    of increasing score.
    """
    scorer = GraspScore(gripper,rigid_object)
    grasps = []
    #TODO... add AntipodalGrasps to the grasps list
    scores = [(scorer.score(g),g) for g in grasps]
    scores = sorted(scores,key = lambda x:x[0])
    if k > len(grasps): k=len(grasps)
    return [(x[1],x[0]) for x in scores[:k]]

#############################################################################

In [ ]:
#Problem 1.C. Run your volume-based grasp sampler.
vis.createWindow()
vis.setViewport(closeup_viewport)
#uncomment this if you want to try your hand at point cloud grasp generation
"""
if world.numRigidObjects() == 1:
    obj_pc = world.makeRigidObject("object1_pc")
    obj_pc.geometry().set(obj.geometry().convert('PointCloud',0.02))
    translation = (0.3,0,0)
    m = obj.getMass()
    m.setCom(vectorops.add(m.getCom(),translation))
    obj_pc.setMass(m)
    obj_pc.geometry().translate(translation)
    obj_pc.appearance().setColor(1,0,1,0.5)
    obj = world.rigidObject(0)
"""
vis.add("world",world)

t0 = time.time()
grasps = antipodal_grasp_sample_volume(known_grippers.robotiq_140,obj,50,2000)
t1 = time.time()
print("Sampled grasps in",t1-t0,"s, min scoring grasp",grasps[0][1])
for i,(g,s) in enumerate(grasps):
    name = "grasp{}".format(i)
    u = math.exp(-(s-grasps[0][1])*2)
    g.add_to_vis(name,(1-u,u,0,1)) 

#uncomment this if you want to try your hand at point cloud grasp generation
"""
t0 = time.time()
grasps = antipodal_grasp_sample_volume(known_grippers.robotiq_140,obj_pc,50,2000)
t1 = time.time()
print("Sampled grasps in",t1-t0,"s, min scoring grasp",grasps[0][1])
for i,(g,s) in enumerate(grasps):
    name = "pc_grasp{}".format(i)
    u = math.exp(-(s-grasps[0][1])*2)
    g.add_to_vis(name,(1-u,u,0,1)) 
"""
vis.show()

D. Define a surface-centric grasp selection strategy in `antipodal_grasp_sample_surface` that samples grasps by sampling points on the object at random.  The approach should generate a list of candidates by sampling two surface points at random, and then construct an AntipodalGrasp through the points.  As before, it should then select the top `k` scoring grasps.

Which approach is more efficient at generating high-quality grasps?  Compare the quality of grasps generated by these techniques, according to your scoring function.  Try running them multiple times to see best-case, worst-case, and average-case quality.  Report how many runs you used, and interpret the results in the written portion at the end of this document.  (Don't worry if your tests are inconclusive -- the best sampler and grasp scores are very much open matters for debate!)

In [ ]:
#Problem 1.D. Run your surface-based grasp sampler.
vis.createWindow()
vis.setViewport(closeup_viewport)
vis.add("world",world)

t0 = time.time()
grasps = antipodal_grasp_sample_surface(known_grippers.robotiq_140,obj,50,2000)
t1 = time.time()
print("Sampled grasps in",t1-t0,"s, min scoring grasp",grasps[0][1])
for i,(g,s) in enumerate(grasps):
    name = "grasp{}".format(i)
    u = math.exp(-(s-grasps[0][1])*2)
    g.add_to_vis(name,(1-u,u,0,1)) 
    
#uncomment this if you want to try your hand at point cloud grasp generation
"""
t0 = time.time()
grasps = antipodal_grasp_sample_surface(known_grippers.robotiq_140,obj_pc,50,2000)
t1 = time.time()
print("Sampled grasps in",t1-t0,"s, min scoring grasp",grasps[0][1])
for i,(g,s) in enumerate(grasps):
    name = "pc_grasp{}".format(i)
    u = math.exp(-(s-grasps[0][1])*2)
    g.add_to_vis(name,(1-u,u,0,1)) 
"""
vis.show()

## Problem 2: Grasp approach planning

A. Write a subroutine that will generate an approach motion for the gripper to attain a grasp. The gripper should start at a finger-open configuration, move forward along its `primary_axis` by `distance` units, and then close its gripper so its fingers are at width `finger_width`. The return value is a movement of the gripper's base link and its finger links stored separately.

To construct the trajectories, you will use the `Trajectory` and `SE3Trajectory` classes. The trajectories consist of a sequence of times `[t0,...,tn]` and  milestones (keyframes) `[m0,...,mn]`, which are then linearly interpolated for you.  The only difference in the SE3Trajectory case is that the interpolation is done in SE3, so that if you do perform rotations they will be interpolated properly with geodesics. Create it using the constructor `SE3Trajectory(times,Ts)` where `Ts` is a list of klampt se3 objects `[T0,...,Tn]`.

The finger configuration at varying opening amounts can be obtained using `GripperInfo.partway_open_config(u)` with `u` in the range $[0,1]$.

In [ ]:
########################## Adaptation of MP2 Problem 2 code ##############################

def match_grasp(gripper_info,grasp):
    """
    Returns:
        (R,t): a Klampt se3 element describing the maching gripper transform
    """
    r_axis = vectorops.cross(gripper_info.secondary_axis,grasp.axis)
    r_angle = math.acos(vectorops.dot(gripper_info.secondary_axis,grasp.axis))
    if r_angle < 1e-5:
        R = so3.identity()
    else:
        r_axis = vectorops.unit(r_axis)
        R = so3.rotation(r_axis,r_angle)
    finger_pos = vectorops.madd(gripper_info.center,gripper_info.primary_axis,gripper_info.finger_length)
    t = vectorops.sub(grasp.center,so3.apply(R,finger_pos))
    return (R,t)

def object_centric_match(gripper_info,grasp_local,obj):
    """Sample a transform for the gripper that meets the desired grasp
    for a RigidObjectModel. 
    
    Args:
        gripper_info (GripperInfo): describes the gripper
        grasp_local (AntipodalGrasp): the desired grasp, with coordinates given in the local frame of
            obj.
        obj (RigidObjectModel): the object to be grasped, posed in the world according to its
            estimated transform.
    
    Returns:
        (R,t): a Klampt se3 element describing a randomly sampled maching gripper transform.
    """
    Tobj = obj.getTransform()
    grasp_world = AntipodalGrasp(se3.apply(Tobj,grasp_local.center),se3.apply_rotation(Tobj,grasp_local.axis))
    
    R0,t0 = match_grasp(gripper_info,grasp_world)
    finger_pos = vectorops.madd(gripper_info.center,gripper_info.primary_axis,gripper_info.finger_length)
    Rrand = so3.rotation(grasp_world.axis,random.uniform(0,math.pi*2))
    return so3.mul(Rrand,R0),vectorops.sub(grasp_world.center,so3.apply(so3.mul(Rrand,R0),finger_pos))

 
####################################################################################

######################### Problem 2 code goes here #################################

def make_grasp_approach(gripper_info,T_grasp,finger_width,distance=None):
    """Given a grasp transform and desired finger width, create a grasp
    approach trajectory that
    1. starts with the fingers open at `distance` units away,
    2. moves toward T_grasp in a straight line along the local approach direction
      `gripper_info.primary_axis`, and then
    3. closes the fingers to the appropriate width.
    
    Args:
        gripper_info (GripperInfo)
        T_grasp (klampt se3 element): the gripper's ultimate transform
        finger_width (float): the desired width between the fingers. Assume the
            gripper linearly interpolates between `gripper_info.minimum_span` and
            `gripper_info.maximum_span`.
        distance (float, optional): the amount the gripper should start away from 
            the object. If None, should be set to `gripper_info.finger_length`
    
    Returns:
        SE3Trajectory,Trajectory: A pair (gripper_traj,finger_traj) describing
        a grasp approach trajectory. gripper_traj gives the Cartesian movement
        of the gripper base, synchronized with the trajectory of the fingers.
        
        The timing of the trajectories can be arbitrary, but they should stay
        synchronized.
    """
    if distance is None:
        distance = gripper_info.finger_length
    fopen = gripper_info.partway_open_config(1)
    #TODO: construct the trajectories
    finger_trajectory = Trajectory([0,1],[fopen,fopen])
    gripper_trajectory = SE3Trajectory([0,1],[T_grasp,T_grasp])
    return gripper_trajectory,finger_trajectory

def sample_grasp_approach(gripper_info,grasp,obj,distance=None):
    """Given an AntipodalGrasp, sample a possible grasp approach trajectory that
    1. starts with the fingers open at `distance` units away,
    2. moves toward T_grasp in a straight line along the local approach direction
      `gripper_info.primary_axis`, and then
    3. closes the fingers to the appropriate width.
    
    Args:
        gripper_info (GripperInfo)
        grasp (AntipodalGrasp): the desired grasp.
        obj (RigidObjectModel):
        distance (float, optional): the amount the gripper should start away from 
            the object. If None, should be set to `gripper_info.finger_length`
    
    Returns:
        SE3Trajectory,Trajectory: A pair (gripper_traj,finger_traj) describing
        a grasp approach trajectory. gripper_traj gives the Cartesian movement
        of the gripper base, synchronized with the trajectory of the fingers.
        
        The timing of the trajectories can be arbitrary, but they should stay
        synchronized.
    """
    Tgrasp = object_centric_match(gripper_info,grasp,obj)
    return make_grasp_approach(gripper_info,Tgrasp,grasp.finger_width,distance)

####################################################################################

In [ ]:
#Problem 2.A. Find a trajectory that meets a grasp
gripper = known_grippers.robotiq_140
world2 = WorldModel()
world2.readFile(gripper.klampt_model)
obj = world2.makeRigidObject(world.rigidObject(0).getName())
obj.geometry().set(world.rigidObject(0).geometry())
obj.appearance().set(world.rigidObject(0).appearance())
obj.setMass(world.rigidObject(0).getMass())
gripper_robot = world2.robot(0)
gripper_geom = gripper.get_geometry(gripper_robot,type='TriangleMesh')

def animate(gripper_traj,finger_traj):
    for t,m in zip(gripper_traj.times,gripper_traj.milestones):
        vis.add("gripper traj "+str(t),gripper_traj.to_se3(m))
    if gripper_traj.startTime() != finger_traj.startTime():
        print("Gripper and finger trajectories not time-aligned")
        return
    if gripper_traj.endTime() != finger_traj.endTime():
        print("Gripper and finger trajectories not time-aligned")
        return
    dt = 1.0/20.0
    N = int(gripper_traj.duration()/dt)
    print("Animating",N,"steps")
    for step in range(N):
        t = step*dt
        T = gripper_traj.eval(t)
        qf = finger_traj.eval(t)
        gripper_robot.link(0).setParentTransform(*T)
        qrobot = gripper_robot.getConfig()
        qrobot_f = gripper.set_finger_config(qrobot,qf)
        gripper_robot.setConfig(qrobot_f)
        vis.update()
        time.sleep(dt)
    print("Done")


def do_wide_grasp():    
    global gripper
    T_test = se3.identity()
    width = 0.1
    gripper_traj,finger_traj = make_grasp_approach(gripper,T_test,width)
    animate(gripper_traj,finger_traj)

grasps = []
    
def do_feas_grasp():
    global gripper,grasps,obj
    if not grasps:
        grasps = antipodal_grasp_sample_surface(gripper,obj,50,2000)
    print("Sampling approach for grasp with score",grasps[0][1])
    gripper_traj,finger_traj = sample_grasp_approach(gripper,grasps[0][0],obj)
    animate(gripper_traj,finger_traj)
    grasps.pop(0)
        
vis.createWindow()
vis.setViewport(closeup_viewport)
vis.add("world",world2)
vis.addAction(do_wide_grasp,"Test make_grasp_approach")
vis.addAction(do_feas_grasp,"Test sample_grasp_approach")
vis.show()

## Inverse kinematics

Now, you will find a configuration for a robot that meets a grasp by solving an inverse kinematics problem. We will use the TRINA robot and the gripper on its left arm for these problems. 

A. The `Sample Transform + IK` function will pick a transform at random, and then call your `solve_robot_ik` function. Implement an inverse kinematics solver using the [klampt.model.ik](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/klampt.model.ik.html#module-klampt.model.ik) functions. You may also need to consult the [IKSolver](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/klampt.robotsim.html?highlight=iksolver#klampt.IKSolver) docs and the [IK manual](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/Manual-IK.html).  If the solver succeeds, the gripper will be colored green.  

Make sure to configure the active degrees of freedom to avoid altering the robot's links 0-5.

B. Observe that many sampled transforms are not reached, either because the transform is unreachable, or the solver simply fell into a local minimum.  The "sample-transform-then-IK" approach fails to take into account that by freezing the transform, the first step causes possible failures in the second step.  As an alternative, you will implement a solver that directly captures the antipodal grasp's axis rotation constraint.  In `sample_grasp_ik`, create a solver that solves an IK problem with a hinge-type constraint. 

Don't forget to calculate the gripper link's transform as the second return value.

Which method is more efficient at generating successful grasping configurations?  Do you think that this will generally be the case? Why or why not?

In [ ]:
######################### Problem 3 code goes here #################################

def solve_robot_ik(robot,gripper,Tgripper):
    """Given a robot, a gripper, and a desired gripper transform,
    solve the IK problem to place the gripper at the desired transform.
    
    Note: do not modify DOFs 0-5.
    
    Returns:
        list or None: Returns None if no solution was found, and
        returns an IK-solving configuration q otherwise.
    
    This function may modify robot.
    
    Args:
        robot (RobotModel)
        gripper (GripperInfo)
        Tgripper (klampt se3 object)
    """
    #TODO: solve the IK problem
    return None

def sample_grasp_ik(robot,gripper,grasp_local,obj):
    """Given a robot, a gripper, a desired antipodal grasp
    (in local coordinates), and an object, solve the IK
    problem to place the gripper at the desired grasp.
    
    Don't forget to set the finger configurations!
    
    Note: do not modify DOFs 0-5.
    
    Returns:
        tuple (q,T): q is None if no solution was found, and
        is an IK-solving configuration q. T is the resulting
        transform of the gripper link.
    
    This function may modify robot.
    
    Args:
        robot (RobotModel)
        gripper (GripperInfo)
        grasp_local (AntipodalGrasp): given in object local coordinates
        obj (RigidObjectModel): the object
    """
    #TODO: solve the IK problem
    return None,None

###################################################################################

In [ ]:
#load the robot
world3 = WorldModel()
gripper = known_grippers.robotiq_140_trina_left
world3.readFile(gripper.klampt_model)
gripper_robot = world3.robot(0)

#set a nice configuration
from klampt.io import resource
resource.setDirectory("../data/resources/TRINA")
qhome = resource.get("home.config")
gripper_robot.setConfig(qhome)

#copy the object from world to world3
obj = world3.makeRigidObject(world.rigidObject(0).getName())
obj.geometry().set(world.rigidObject(0).geometry())
obj.appearance().set(world.rigidObject(0).appearance())
obj.setMass(world.rigidObject(0).getMass())
obj.setTransform(so3.identity(),[1,0,0.8])

def sample_transform_and_solve_ik():
    global gripper,gripper_robot,grasps,obj
    if not grasps:
        Tobj = obj.getTransform()
        obj.setTransform(*se3.identity())
        grasps = antipodal_grasp_sample_surface(gripper,obj,50,2000)
        obj.setTransform(*Tobj)
    gripper_traj,finger_traj = sample_grasp_approach(gripper,grasps[0][0],obj)
    Ttarget = gripper_traj.to_se3(gripper_traj.milestones[-1])
    vis.add("target xform",Ttarget)
    qrob = solve_robot_ik(gripper_robot,gripper,Ttarget)
    if qrob is not None:
        #inject the finger configuration
        qrob = gripper.set_finger_config(qrob,finger_traj.milestones[-1])
        #set the robot configuration
        gripper_robot.setConfig(qrob)
        vis.setColor(vis.getItemName(gripper_robot.link(gripper.base_link)),0,1,0)
    else:
        vis.setColor(vis.getItemName(gripper_robot.link(gripper.base_link)),1,0,1,1)
    vis.update()
    grasps.pop(0)

def sample_ik_direct():
    global gripper,gripper_robot,grasps,obj
    if not grasps:
        Tobj = obj.getTransform()
        obj.setTransform(*se3.identity())
        grasps = antipodal_grasp_sample_surface(gripper,obj,50,2000)
        obj.setTransform(*Tobj)
    q_robot,Tgripper = sample_grasp_ik(gripper_robot,gripper,grasps[0][0],obj)
    if q_robot is not None:
        vis.add("target xform",Tgripper)
        #set the robot configuration
        gripper_robot.setConfig(q_robot)
        vis.setColor(vis.getItemName(gripper_robot.link(gripper.base_link)),0,1,0)
    else:
        vis.setColor(vis.getItemName(gripper_robot.link(gripper.base_link)),1,0,1,1)
    vis.update()
    grasps.pop(0)

    
vis.createWindow()
vis.addAction(sample_transform_and_solve_ik,"Sample transform + IK")
vis.addAction(sample_ik_direct,"Sample IK")
vis.add("world",world3)
vis.show()

### Problem 3.C *Only for Section IR2*

Modify your `solve_robot_ik` and `sample_grasp_ik` functions to only return successfully if the resulting configuration is collision-free. (Ignore collisions between the object and the finger links, which are named `left_gripper:right_inner_finger_pad` and `left_gripper:left_inner_finger_pad`).  The `Geometry3D.collides` function and the `RobotModel.selfCollides` function will be helpful here.  

Furthermore, implement a random-restart technique with 20 restarts to increase the likelihood of sampling a collision-free, IK-solving configuration.



## Written answers

### 1.D
Place your answers to the written portion of 1.D here

### 3.B
Place your answers to the written portion of 3.B here